In [4]:
import pickle
import joblib

import numpy as np
import pandas as pd

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv(r"C:\Users\manue\OneDrive\Escritorio\Ironhack\Proyectos\Final-Project\Data\Fraud.csv\Fraud.csv")
df.columns = [col.lower().replace(' ', '_') for col in df.columns]
df.drop(columns='nameorig', inplace=True)

le = LabelEncoder()
df['type'] = le.fit_transform(df['type'])
df['namedest'] = le.fit_transform(df['namedest'])

X = df.drop(columns='isfraud')
y = df['isfraud']

over_sample = SMOTE(random_state=0)
X,y = over_sample.fit_resample(X,y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)



joblib.dump(xgb_model, "xgb_weights.pkl")    
df.to_csv(r"C:\Users\manue\OneDrive\Escritorio\Ironhack\Proyectos\Final-Project\Data wrangling\df.csv", index=False)

In [2]:
import joblib

In [3]:
joblib.dump(weights, "xgb_weights.pkl")

NameError: name 'weights' is not defined